In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from sklearn.model_selection import train_test_split
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/xerces-1.3.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xerces,1.3.0,org.apache.xerces.readers.StreamingCharReader,34,2,0,14,74,0,1,...,1.0,2,0.179487,0.160839,1,3,83.558824,2,1.0588,0
1,xerces,1.3.0,org.apache.xml.serialize.OutputFormat,46,1,0,13,61,833,11,...,1.0,1,0.000000,0.220497,0,0,11.195652,12,1.6522,0
2,xerces,1.3.0,org.w3c.dom.html.HTMLFrameSetElement,4,1,0,0,4,6,0,...,0.0,0,0.000000,0.750000,0,0,0.000000,1,1.0000,0
3,xerces,1.3.0,org.w3c.dom.html.HTMLLIElement,4,1,0,0,4,6,0,...,0.0,0,0.000000,0.500000,0,0,0.000000,1,1.0000,0
4,xerces,1.3.0,org.apache.xerces.validators.dtd.DTDGrammar,41,2,0,11,75,280,2,...,1.0,10,0.487179,0.306620,1,7,28.414634,3,1.1951,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,xerces,1.3.0,org.xml.sax.ErrorHandler,3,1,0,0,3,3,0,...,0.0,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
449,xerces,1.3.0,org.apache.xerces.readers.XMLCatalogHandler,11,1,1,1,18,15,1,...,1.0,0,0.000000,0.431818,0,0,8.090909,1,0.9091,0
450,xerces,1.3.0,org.apache.xerces.utils.IntStack,8,1,0,2,14,0,2,...,1.0,0,0.000000,0.625000,0,0,16.750000,4,1.3750,0
451,xerces,1.3.0,org.apache.xerces.msg.SchemaMessages,3,3,0,0,4,1,0,...,0.0,0,0.974359,1.000000,0,0,159.000000,1,0.3333,4


In [4]:
train, test = train_test_split(ant_master, test_size=0.2)

In [5]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [6]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
46,11,4,0,3,14,55,0,3,11,2.000000,61,0.00000,0,0.891304,0.545455,1,4,4.545455,1,0.9091
343,2,1,0,2,2,1,1,1,2,2.000000,2,0.00000,0,0.000000,0.750000,0,0,0.000000,1,1.0000
78,81,1,0,37,257,1504,3,36,9,0.788672,10701,0.96875,10,0.000000,0.135526,0,0,130.716049,25,1.5432
207,4,1,0,2,4,6,1,1,4,2.000000,4,0.00000,0,0.000000,0.750000,0,0,0.000000,1,1.0000
73,0,1,0,1,0,0,1,0,0,2.000000,0,0.00000,0,0.000000,0.000000,0,0,0.000000,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,2,1,0,2,2,1,1,1,2,2.000000,2,0.00000,0,0.000000,0.750000,0,0,0.000000,1,1.0000
425,1,4,0,0,2,0,0,0,1,2.000000,5,0.00000,0,1.000000,1.000000,0,0,4.000000,0,0.0000
63,9,1,0,11,26,16,1,10,9,0.875000,191,1.00000,0,0.000000,0.333333,0,0,19.666667,2,1.0000
308,11,1,0,40,16,25,40,0,11,0.050000,191,0.00000,0,0.000000,0.409091,1,1,16.000000,7,1.1818


In [7]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
178,2,3,0,6,4,1,6,0,2,2.000000,9,0.000000,0,1.000000,0.750000,0,0,3.500000,0,0.0000
103,6,1,0,0,6,15,0,0,6,2.000000,6,0.000000,0,0.000000,0.750000,0,0,0.000000,1,1.0000
404,8,1,0,0,8,28,0,0,8,2.000000,8,0.000000,0,0.000000,0.354167,0,0,0.000000,1,1.0000
326,2,1,0,0,2,1,0,0,2,2.000000,2,0.000000,0,0.000000,0.750000,0,0,0.000000,1,1.0000
0,34,2,0,14,74,0,1,14,21,0.672727,2890,1.000000,2,0.179487,0.160839,1,3,83.558824,2,1.0588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,0,1,0,2,0,0,1,1,0,2.000000,0,0.000000,0,0.000000,0.000000,0,0,0.000000,0,0.0000
114,5,1,0,2,14,0,1,1,4,1.243808,1114,0.009288,0,0.000000,0.500000,0,0,157.200000,9,2.2000
259,25,4,0,3,32,300,0,3,25,2.000000,142,0.000000,0,0.773585,0.312000,1,7,4.680000,1,0.9600
302,6,2,0,7,20,9,1,7,3,0.000000,136,1.000000,0,0.545455,0.433333,1,2,21.500000,5,1.5000


In [8]:
y_train = train['bug']
y_test = test['bug']

In [9]:
y_train

46      0
343     0
78     30
207     0
73      0
       ..
340     0
425     0
63      0
308     0
424     0
Name: bug, Length: 362, dtype: int64

In [10]:
y_test

178    0
103    0
404    0
326    0
0      0
      ..
113    0
114    1
259    0
302    0
96     0
Name: bug, Length: 91, dtype: int64

In [11]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [12]:
x_train_np

array([[ 11.        ,   4.        ,   0.        , ...,   4.54545455,
          1.        ,   0.9091    ],
       [  2.        ,   1.        ,   0.        , ...,   0.        ,
          1.        ,   1.        ],
       [ 81.        ,   1.        ,   0.        , ..., 130.7160494 ,
         25.        ,   1.5432    ],
       ...,
       [  9.        ,   1.        ,   0.        , ...,  19.66666667,
          2.        ,   1.        ],
       [ 11.        ,   1.        ,   0.        , ...,  16.        ,
          7.        ,   1.1818    ],
       [  6.        ,   1.        ,   0.        , ...,  13.33333333,
          1.        ,   0.6667    ]])

In [13]:
x_test_np

array([[ 2.  ,  3.  ,  0.  , ...,  3.5 ,  0.  ,  0.  ],
       [ 6.  ,  1.  ,  0.  , ...,  0.  ,  1.  ,  1.  ],
       [ 8.  ,  1.  ,  0.  , ...,  0.  ,  1.  ,  1.  ],
       ...,
       [25.  ,  4.  ,  0.  , ...,  4.68,  1.  ,  0.96],
       [ 6.  ,  2.  ,  0.  , ..., 21.5 ,  5.  ,  1.5 ],
       [ 5.  ,  4.  ,  0.  , ...,  4.6 ,  1.  ,  0.8 ]])

In [14]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [15]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [16]:
train_x

array([[[ 11.        ,   4.        ,   0.        , ...,   4.54545455,
           1.        ,   0.9091    ]],

       [[  2.        ,   1.        ,   0.        , ...,   0.        ,
           1.        ,   1.        ]],

       [[ 81.        ,   1.        ,   0.        , ..., 130.7160494 ,
          25.        ,   1.5432    ]],

       ...,

       [[  9.        ,   1.        ,   0.        , ...,  19.66666667,
           2.        ,   1.        ]],

       [[ 11.        ,   1.        ,   0.        , ...,  16.        ,
           7.        ,   1.1818    ]],

       [[  6.        ,   1.        ,   0.        , ...,  13.33333333,
           1.        ,   0.6667    ]]])

In [17]:
test_y

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 3, 0, 1, 0, 0, 0, 1,
       0, 0, 0])

In [18]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [19]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])

Epoch 1/100
4/4 [==============================] - 4s 60ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4699 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 2/100
4/4 [==============================] - 0s 8ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 3/100
4/4 [==============================] - 0s 7ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 4/100
4/4 [==============================] - 0s 8ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 5/100
4/4 [==============================] - 0s 9ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 6/100
4/4 [==============================] - 0s 6ms/step - loss: 4.5856 - mse: 4.5856 - 

4/4 [==============================] - 0s 8ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 46/100
4/4 [==============================] - 0s 7ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 47/100
4/4 [==============================] - 0s 7ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 48/100
4/4 [==============================] - 0s 7ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 49/100
4/4 [==============================] - 0s 7ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 50/100
4/4 [==============================] - 0s 7ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4

4/4 [==============================] - 0s 9ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 91/100
4/4 [==============================] - 0s 11ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 92/100
4/4 [==============================] - 0s 10ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 93/100
4/4 [==============================] - 0s 7ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 94/100
4/4 [==============================] - 0s 6ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0.4696 - root_mean_squared_error: 2.1414 - mean_squared_logarithmic_error: 0.2575
Epoch 95/100
4/4 [==============================] - 0s 9ms/step - loss: 4.5856 - mse: 4.5856 - mae: 0

In [20]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 3750), started 3:00:38 ago. (Use '!kill 3750' to kill it.)

In [21]:
score = model.evaluate(test_x, test_y)
score

3/3 [==============================] - 1s 3ms/step - loss: 0.5824 - mse: 0.5824 - mae: 0.2527 - root_mean_squared_error: 0.7632 - mean_squared_logarithmic_error: 0.1474


[0.5824176073074341,
 0.5824176073074341,
 0.2527472674846649,
 0.7631629109382629,
 0.14744976162910461]